In [5]:
%AddDeps org.deeplearning4j deeplearning4j-core 0.6.0
%AddDeps com.databricks spark-csv_2.10 1.4.0 --transitive

Marking org.deeplearning4j:deeplearning4j-core:0.6.0 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps7974131878688600591/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps7974131878688600591/https/repo1.maven.org/maven2/org/deeplearning4j/deeplearning4j-core/0.6.0/deeplearning4j-core-0.6.0.jar
Marking com.databricks:spark-csv_2.10:1.4.0 for download
Preparing to fetch from:
-> file:/tmp/toree_add_deps7974131878688600591/
-> https://repo1.maven.org/maven2
-> New file at /tmp/toree_add_deps7974131878688600591/https/repo1.maven.org/maven2/com/databricks/spark-csv_2.10/1.4.0/spark-csv_2.10-1.4.0.jar
-> New file at /tmp/toree_add_deps7974131878688600591/https/repo1.maven.org/maven2/com/univocity/univocity-parsers/1.5.1/univocity-parsers-1.5.1.jar
-> New file at /tmp/toree_add_deps7974131878688600591/https/repo1.maven.org/maven2/org/apache/commons/commons-csv/1.1/commons-csv-1.1.jar


# Spark gradient boosting

In [ ]:
/*import org.apache.spark.mllib.tree.GradientBoostedTrees
import org.apache.spark.mllib.tree.configuration.BoostingStrategy
import org.apache.spark.mllib.tree.model.GradientBoostedTreesModel
import org.apache.spark.mllib.util.MLUtils

// Load and parse the data file.
val data = MLUtils.loadLibSVMFile(sc, "data/mllib/sample_libsvm_data.txt")
// Split the data into training and test sets (30% held out for testing)
val splits = data.randomSplit(Array(0.7, 0.3))
val (trainingData, testData) = (splits(0), splits(1))

// Train a GradientBoostedTrees model.
// The defaultParams for Regression use SquaredError by default.
val boostingStrategy = BoostingStrategy.defaultParams("Regression")
boostingStrategy.numIterations = 3 // Note: Use more iterations in practice.
boostingStrategy.treeStrategy.maxDepth = 5
// Empty categoricalFeaturesInfo indicates all features are continuous.
boostingStrategy.treeStrategy.categoricalFeaturesInfo = Map[Int, Int]()

val model = GradientBoostedTrees.train(trainingData, boostingStrategy)

// Evaluate model on test instances and compute test error
val labelsAndPredictions = testData.map { point =>
  val prediction = model.predict(point.features)
  (point.label, prediction)
}
val testMSE = labelsAndPredictions.map{ case(v, p) => math.pow((v - p), 2)}.mean()
println("Test Mean Squared Error = " + testMSE)
println("Learned regression GBT model:\n" + model.toDebugString)

// Save and load model
model.save(sc, "target/tmp/myGradientBoostingRegressionModel")
val sameModel = GradientBoostedTreesModel.load(sc,
  "target/tmp/myGradientBoostingRegressionModel")*/

# Read in the data

In [3]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
val sqlContext = SQLContext.getOrCreate(sc)
import sqlContext.implicits._

In [4]:
val df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load("./dat/train.csv")

In [8]:
df.select("id", "cat1", "cat2").show

+---+----+----+
| id|cat1|cat2|
+---+----+----+
|  1|   A|   B|
|  2|   A|   B|
|  5|   A|   B|
| 10|   B|   B|
| 11|   A|   B|
| 13|   A|   B|
| 14|   A|   A|
| 20|   A|   B|
| 23|   A|   B|
| 24|   A|   B|
| 25|   A|   B|
| 33|   A|   B|
| 34|   B|   A|
| 41|   B|   A|
| 47|   A|   A|
| 48|   A|   A|
| 49|   A|   B|
| 51|   A|   A|
| 52|   A|   A|
| 55|   A|   A|
+---+----+----+
only showing top 20 rows



In [13]:
val (catCols, contCols) = df.columns.partition(_.contains("cat"))

In [16]:
print(catCols.length)
catCols

116

Array(cat1, cat2, cat3, cat4, cat5, cat6, cat7, cat8, cat9, cat10, cat11, cat12, cat13, cat14, cat15, cat16, cat17, cat18, cat19, cat20, cat21, cat22, cat23, cat24, cat25, cat26, cat27, cat28, cat29, cat30, cat31, cat32, cat33, cat34, cat35, cat36, cat37, cat38, cat39, cat40, cat41, cat42, cat43, cat44, cat45, cat46, cat47, cat48, cat49, cat50, cat51, cat52, cat53, cat54, cat55, cat56, cat57, cat58, cat59, cat60, cat61, cat62, cat63, cat64, cat65, cat66, cat67, cat68, cat69, cat70, cat71, cat72, cat73, cat74, cat75, cat76, cat77, cat78, cat79, cat80, cat81, cat82, cat83, cat84, cat85, cat86, cat87, cat88, cat89, cat90, cat91, cat92, cat93, cat94, cat95, cat96, cat97, cat98, cat99, cat100, cat101, cat102, cat103, cat104, cat105, cat106, cat107, cat108, cat109, cat1...

In [19]:
print(contCols.length)
contCols

16

Array(id, cont1, cont2, cont3, cont4, cont5, cont6, cont7, cont8, cont9, cont10, cont11, cont12, cont13, cont14, loss)